# AutoSDF evaluation

In [6]:
import trimesh
import numpy as np
import os
import pandas as pd
from sklearn.metrics import pairwise_distances
from typing import List, Dict

# Define input and output directories
input_folder = r"/home/student/Documents/Data/Pre-processed_data/windows-preprocessed/synthetisch_45Y_v2"
results_folder = r"/home/student/Documents/Data/AutoSDF_results/Windows-preprocessed/synthetisch_results_45Y_v2"
output_csv = r"/home/student/Documents/Data/Evaluation_results/evaluatie_45Y_tot10.csv"


## Meervoudige IOU

In [19]:
# Berekeneningen meervoudig
def evaluate_mesh_similarity(mesh1: trimesh.Trimesh, mesh2: trimesh.Trimesh, num_samples: int = 10000, threshold: float = 0.01) -> dict:
    volume_iou = None
    hollow_iou = None
    convex_hull_iou = None

    mesh1.fix_normals()
    mesh2.fix_normals()
    mesh2.apply_scale(2)

    # Volume_IOU
    try:
        if mesh1.is_watertight and mesh2.is_watertight:
            intersection = mesh1.intersection(mesh2)
            union = mesh1.union(mesh2)
            if intersection.volume > 0 and union.volume > 0:
                volume_iou = intersection.volume / union.volume
            else: print("volumes are negative: ", intersection.volume, union.volume)
        else: print("Meshes are not watertight: mesh1: ", str(mesh1.is_watertight),", mesh2: ", str(mesh2.is_watertight) )
    except Exception as e:
        print(f"Fout in volume_iou: {e}")

    # hollow_iou
    try:
        hull1 = mesh1.convex_hull
        hull2 = mesh2.convex_hull
        outer_volume1 = hull1.volume
        outer_volume2 = hull2.volume
        inner_volume1 = mesh1.volume - outer_volume1
        inner_volume2 = mesh2.volume - outer_volume2
        intersection_inner = min(inner_volume1, inner_volume2)
        union_inner = max(inner_volume1, inner_volume2)
        intersection_outer = min(outer_volume1, outer_volume2)
        union_outer = max(outer_volume1, outer_volume2)
        iou_inner = intersection_inner / union_inner if union_inner > 0 else 0
        iou_outer = intersection_outer / union_outer if union_outer > 0 else 0
        hollow_iou = (iou_inner + iou_outer) / 2
    except Exception as e:
        print(f"Fout in hollow_iou: {e}")

    try:
        # Chamfer Distance
        points1 = mesh1.vertices
        points2 = mesh2.vertices
        distances_1to2 = pairwise_distances(points1, points2)
        distances_2to1 = pairwise_distances(points2, points1)

        #points1, face_indices1 = mesh1.sample(num_samples, return_index=True)
        #points2, face_indices2 = mesh2.sample(num_samples, return_index=True)
        #distances_1to2 = pairwise_distances(points1, points2)
        #distances_2to1 = pairwise_distances(points2, points1)
        chamfer_1 = np.mean(np.min(distances_1to2, axis=1)**2)
        chamfer_2 = np.mean(np.min(distances_2to1, axis=1)**2)
        chamfer = chamfer_1 + chamfer_2

        # F-score
        precision = np.mean(np.min(distances_1to2, axis=1) < threshold)
        recall = np.mean(np.min(distances_2to1, axis=1) < threshold)
        fscore = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Coverage
        coverage = precision

        # Normal consistancy
        # Assume normals are per vertex
        normals1 = mesh1.vertex_normals
        normals2 = mesh2.vertex_normals
        # Closest indices from mesh1 to mesh2
        indices_1to2 = np.argmin(distances_1to2, axis=1)
        matched_normals2 = normals2[indices_1to2]
        normal_dot_products = np.einsum('ij,ij->i', normals1, matched_normals2)
        normal_dot_products = np.clip(normal_dot_products, -1.0, 1.0)
        normal_cons = np.mean(np.abs(normal_dot_products))  # Cosine similarity

        #normals1 = mesh1.vertex_normals[vertex_indices1]
        #normals1 = mesh1.face_normals[face_indices1]
        #normals2 = mesh2.face_normals[face_indices2]
        #dot_products = np.einsum('ij,ij->i', normals1, normals2)
        #dot_products = np.clip(dot_products, -1.0, 1.0)
        #normal_cons = np.mean(np.abs(dot_products))
    except Exception as e:
        print(f"Fout in metriekberekening: {e}")
        chamfer = fscore = coverage = normal_cons = None

    return {
        "volume_iou": volume_iou,
        "hollow_iou": hollow_iou,
        "chamfer_distance": chamfer,
        "f_score": fscore,
        "mesh_coverage": coverage,
        "normal_consistency": normal_cons
    }

def split_path(path: str) -> List[str]:
    return path.split(os.sep)

In [20]:
from typing import Union


def process_meshes(input_folder: str, results_folder: str, output_csv: str) -> None:
  
    results: List[List[Union[str, float]]] = []  # Resultatenlijst met de juiste types
    input_meshes: Dict[str, str] = {}

    # Verzamel alle .obj-bestanden in de inputfolder (veronderstelt dat ze gt.obj zijn)
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file == "sdf.obj":
                rel_path = os.path.relpath(os.path.join(root, file), input_folder)
                input_meshes[rel_path] = os.path.join(root, file)
    #i = 0
    print(len(input_meshes.items()))
    # Vergelijk elk input mesh (gt.obj) met de bijbehorende meshes in de resultsfolder
    for rel_path, input_path in input_meshes.items():
        #if i > 10: return
        #i+=1
        # Verkrijg de naam van de submap
        result_subfolder = os.path.join(results_folder, os.path.dirname(rel_path))
        
        # Controleer of er een overeenkomstige subfolder in de results folder bestaat
        if os.path.exists(result_subfolder):
            for result_root, _, result_files in os.walk(result_subfolder):
                for result_file in result_files:
                    # Zorg ervoor dat we alleen de juiste outputmeshes vergelijken (0.obj, 1.obj, 2.obj)
                    if result_file.endswith(".obj") and result_file in ["0.obj", "1.obj", "2.obj"]:
                        result_path: str = os.path.join(result_root, result_file)

                        # Laad de meshes
                        mesh1 = trimesh.load_mesh(input_path)
                        mesh2 = trimesh.load_mesh(result_path)
                        
                        # Verkrijg de vergelijkingsresultaten via evaluate_mesh_similarity
                        comparison_results = evaluate_mesh_similarity(mesh1, mesh2)
                        
                        # Maak een lijst met het rel_path (input), result_path (output) en de metrieken
                        rel_result_path = os.path.relpath(result_path, results_folder)
                        input_hierarchy = split_path(rel_path)
                        result_folder = os.path.basename(os.path.dirname(rel_result_path))
                        result_filename = os.path.basename(rel_result_path)
                        
                        # Voeg alle resultaten toe aan de lijst
                        result_row = input_hierarchy + [result_folder, result_filename] + list(comparison_results.values())
                        results.append(result_row)
                        
                        print(result_row)  # Debugging: Laat de laatste rij zien

    # Controleer of de results lijst leeg is voordat we doorgaan
    if not results:
        print("Geen resultaten om te verwerken. Controleer de input- en results-folders.")
        return  # Stop de functie als er geen resultaten zijn.

    # Bepaal de kolomnamen van de CSV
    max_depth = max(len(row) for row in results)
    column_names = [f"Level {i+1}" for i in range(max_depth - 8)] + [
        'Result Folder',
        'Result File',
        'Volume IoU',
        'Hollow IoU',
        'Chamfer Distance',
        'Normal Consistency',
        'F-score',
        'Mesh Coverage'
    ]
    
    # Zet de resultaten om naar een DataFrame
    df = pd.DataFrame(results, columns=column_names)

    # Sla het resultaat op in een CSV bestand
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Call the function to process the meshes
process_meshes(input_folder, results_folder, output_csv)

59
Meshes are not watertight: mesh1:  True , mesh2:  False
['buis_recht', 'recht_bocht_180', 'sdf.obj', 'voxelData_schijf_links_4', '1.obj', None, 0.37954491518700123, 0.036688029920919046, 0.04882133287459401, 0.04800759013282732, 0.6327726215670308]
['buis_recht', 'recht_bocht_180', 'sdf.obj', 'voxelData_schijf_links_4', '0.obj', 0.4731578843501816, 0.38459820373658377, 0.03512284678528954, 0.04071202954998051, 0.039468690702087285, 0.6327426122956962]
['buis_recht', 'recht_bocht_180', 'sdf.obj', 'voxelData_schijf_links_4', '2.obj', 0.48959007059475784, 0.39646428428794617, 0.033787644070599115, 0.047869492374389006, 0.04573055028462998, 0.6521025736140941]
['buis_recht', 'recht_bocht_180', 'sdf.obj', 'voxelData_schijf_onder_4', '1.obj', 0.5764278353348443, 0.25482013028960715, 0.023186579666364438, 0.023360369218876995, 0.028462998102466792, 0.8100300581000818]
['buis_recht', 'recht_bocht_180', 'sdf.obj', 'voxelData_schijf_onder_4', '0.obj', 0.7239264849695904, 0.4037064434352877, 0

KeyboardInterrupt: 

## Gewone IOU

In [ ]:
# Berekeneningen eenvoudig

def evaluate_mesh_similarity(mesh1: trimesh.Trimesh, mesh2: trimesh.Trimesh, num_samples: int = 10000, threshold: float = 0.01) -> dict:
    try:
        # Controleer of de meshes een geldig volume hebben
        if not mesh1.is_volume or not mesh2.is_volume:
            raise ValueError("Een van de meshes heeft geen geldig volume.")

        # Probeer de intersection en union
        intersection = mesh1.intersection(mesh2)
        union = mesh1.union(mesh2)

        # Controleer of de intersection en union volumes geldig zijn
        if intersection.volume <= 0 or union.volume <= 0:
            raise ValueError(f"Intersection volume: {intersection.volume}, Union volume: {union.volume}")
        
        iou = intersection.volume / union.volume if union.volume > 0 else 0

    except Exception as e:
        print(f"Fout bij het berekenen van IoU: {e}")
        iou = -1

    # Sample punten en normale indices
    points1, face_indices1 = mesh1.sample(num_samples, return_index=True)
    points2, face_indices2 = mesh2.sample(num_samples, return_index=True)

    # Chamfer Distance
    distances_1to2 = pairwise_distances(points1, points2)
    distances_2to1 = pairwise_distances(points2, points1)
    chamfer_1 = np.mean(np.min(distances_1to2, axis=1))
    chamfer_2 = np.mean(np.min(distances_2to1, axis=1))
    chamfer = chamfer_1 + chamfer_2

    # F-score
    precision = np.mean(np.min(distances_1to2, axis=1) < threshold)
    recall = np.mean(np.min(distances_2to1, axis=1) < threshold)
    fscore = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Mesh coverage
    coverage = precision  

    # Normal consistency
    normals1 = mesh1.face_normals[face_indices1]
    normals2 = mesh2.face_normals[face_indices2]
    dot_products = np.einsum('ij,ij->i', normals1, normals2)
    dot_products = np.clip(dot_products, -1.0, 1.0)
    normal_cons = np.mean(np.abs(dot_products))

    return {
        "iou": iou,
        "chamfer_distance": chamfer,
        "f_score": fscore,
        "mesh_coverage": coverage,
        "normal_consistency": normal_cons
    }

def split_path(path: str) -> List[str]:
    return path.split(os.sep)

In [ ]:
def process_meshes(input_folder: str, results_folder: str, output_csv: str) -> None:
  
    results: List[List[Union[str, float]]] = []  # Resultatenlijst met de juiste types
    input_meshes: Dict[str, str] = {}

    # Verzamel alle .obj-bestanden in de inputfolder (veronderstelt dat ze gt.obj zijn)
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file == "gt.obj":
                rel_path = os.path.relpath(os.path.join(root, file), input_folder)
                input_meshes[rel_path] = os.path.join(root, file)

    # Vergelijk elk input mesh (gt.obj) met de bijbehorende meshes in de resultsfolder
    for rel_path, input_path in input_meshes.items():
        # Verkrijg de naam van de submap
        result_subfolder = os.path.join(results_folder, os.path.dirname(rel_path))
        
        # Controleer of er een overeenkomstige subfolder in de results folder bestaat
        if os.path.exists(result_subfolder):
            for result_root, _, result_files in os.walk(result_subfolder):
                for result_file in result_files:
                    # Zorg ervoor dat we alleen de juiste outputmeshes vergelijken (0.obj, 1.obj, 2.obj)
                    if result_file.endswith(".obj") and result_file in ["0.obj", "1.obj", "2.obj"]:
                        result_path: str = os.path.join(result_root, result_file)

                        # Laad de meshes
                        mesh1 = trimesh.load_mesh(input_path)
                        mesh2 = trimesh.load_mesh(result_path)
                        
                        # Verkrijg de vergelijkingsresultaten via evaluate_mesh_similarity
                        comparison_results = evaluate_mesh_similarity(mesh1, mesh2)
                        
                        # Maak een lijst met het rel_path (input), result_path (output) en de metrieken
                        rel_result_path = os.path.relpath(result_path, results_folder)
                        input_hierarchy = split_path(rel_path)
                        result_folder = os.path.basename(os.path.dirname(rel_result_path))
                        result_filename = os.path.basename(rel_result_path)
                        
                        # Voeg alle resultaten toe aan de lijst
                        result_row = input_hierarchy + [result_folder, result_filename] + list(comparison_results.values())
                        results.append(result_row)
                        
                        print(result_row)  # Debugging: Laat de laatste rij zien

    # Controleer of de results lijst leeg is voordat we doorgaan
    if not results:
        print("Geen resultaten om te verwerken. Controleer de input- en results-folders.")
        return  # Stop de functie als er geen resultaten zijn.

    # Bepaal de kolomnamen van de CSV
    max_depth = max(len(row) for row in results)
    column_names = [f"Level {i+1}" for i in range(max_depth - 5)] + ['Result Folder', 'Result File', 'IoU', 'Chamfer Distance', 'Normal Consistency', 'F-score', 'Mesh Coverage']
    
    # Zet de resultaten om naar een DataFrame
    df = pd.DataFrame(results, columns=column_names)

    # Sla het resultaat op in een CSV bestand
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Call the function to process the meshes
process_meshes(input_folder, results_folder, output_csv)

### test Intersections

In [23]:
import trimesh

input_path_sdf = r"/home/student/Documents/Data/Pre-processed_data/windows-preprocessed/synthetisch_45Y_v2/buis_recht/recht_bocht_15/sdf.obj"
input_path_gt = r"/home/student/Documents/Data/Pre-processed_data/windows-preprocessed/synthetisch_45Y_v2/buis_recht/recht_bocht_15/sdf.obj"

result_path = r"/home/student/Documents/Data/AutoSDF_results/Windows-preprocessed/synthetisch_results_45Y_v2/buis_recht/recht_bocht_15/voxelData_8e_onder_4/0.obj"

# Laad de meshes
mesh1 = trimesh.load_mesh(input_path_sdf)
mesh2 = trimesh.load_mesh(result_path)
mesh1.fix_normals()
mesh2.fix_normals()
mesh2.apply_scale(2)
mesh1.show()
mesh2.show()

In [22]:

# Volume_IOU
try:
    if mesh1.is_watertight and mesh2.is_watertight:
        print(mesh1.volume, mesh2.volume)
        intersection = mesh1.intersection(mesh2)
        print(intersection.volume)
        union = mesh1.union(mesh2)
        print(union.volume)
        if intersection.volume > 0 and union.volume > 0:
            volume_iou = intersection.volume / union.volume
            print(volume_iou)
except Exception as e:
    print(f"Fout in volume_iou: {e}")

0.2601995098985445 0.3375983123538984
0.25372304089919145
0.34407478122124774
0.7374066765330352
